2. Обучим модель, которая будет извлекать автомобильные сущности.

In [14]:
pip install deeppavlov

In [15]:
pip install tensorflow==1.15.0

In [16]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [17]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-cpczyvho
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-cpczyvho
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23593 sha256=5a0a744906dba95e899a5a6b7cc591b7205fb3a84cb214ddfeff3b3ae6971232
  Stored in directory: /tmp/pip-ephem-wheel-cache-kh9wwum_/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [18]:
import pandas as pd

In [19]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = '/'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = '/'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]

In [20]:
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

2021-06-17 19:58:18.822 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz download because of matching hashes
2021-06-17 19:58:39.580 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2021-06-17 19:58:40.10 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]
2021-06-17 19:59:13.952 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model


In [21]:
import pandas as pd

In [22]:
with open('/content/drive/MyDrive/Colab_Notebooks/avito_cars.txt') as f2: cars = f2.readlines()
data = pd.DataFrame({'text': cars})

In [23]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [24]:
marked

[[('Ваз', 'B-PRODUCT'),
  ('2107', 'I-PRODUCT'),
  ('в', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  (',', 'O'),
  ('полностью', 'O'),
  ('на', 'O'),
  ('ходу', 'O'),
  (',', 'O'),
  ('коробка', 'O'),
  ('5ст', 'B-CARDINAL'),
  (',', 'O'),
  ('Центральный', 'O'),
  ('замок', 'O'),
  (',', 'O'),
  ('музыка', 'O'),
  (',', 'O'),
  ('хорошая', 'O'),
  ('летняя', 'O'),
  ('резина', 'O'),
  ('.', 'O'),
  ('торг', 'O'),
  ('уместен', 'O'),
  ('.', 'O'),
  ('/', 'O'),
  ('/', 'O'),
  ('Ваз', 'O'),
  ('2109', 'B-CARDINAL'),
  ('1990г', 'B-DATE'),
  ('.', 'I-DATE'),
  ('в', 'I-DATE'),
  ('.', 'O'),
  ('В', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  ('на', 'O'),
  ('полном', 'O'),
  ('ходу', 'O'),
  (',', 'O'),
  ('салон', 'O'),
  ('ухоженный', 'O'),
  (',', 'O'),
  ('передние', 'O'),
  ('сиденья', 'O'),
  ('на', 'O'),
  ('пульте', 'O'),
  ('управления', 'O'),
  ('.', 'O'),
  ('Центральный', 'O'),
  ('замок', 'O'),
  (',', 'O'),
  ('музыка', 'O'),
  (',', 'O'),
  ('торг', 'O'),
  

а) 3 примера сущностей, которых не было в разметке, но которые правильно разметились моделью: 

1. ('KIA', 'B-PRODUCT'), ('BONGO', 'I-PRODUCT'), ('III', 'I-PRODUCT'),
2. ('ВАЗ', 'B-PRODUCT'), ('211440', 'I-PRODUCT'),
3. ('ГАЗ', 'B-PRODUCT'),('330202', 'I-PRODUCT'),
4. ('Тойота', 'B-PRODUCT'), ('Ланд', 'I-PRODUCT'), ('Крузер', 'I-PRODUCT'), ('Прадо', 'I-PRODUCT'), ('120', 'I-PRODUCT')

б) 5 ошибок в работе модели:

1. Определил модель и бренд как ORG - ('KIA', 'B-ORG') ('Cerato', 'I-ORG'), ('HONDA', 'B-ORG') ('FIT', 'I-ORG')
2. Определил модель и бренд как PERSON - ('Рено', 'B-PERSON') ('Симбол', 'I-PERSON')
3. Определил тире как чать названия машины (но может и правильно) ('-', 'I-PRODUCT')
4. Определил год как модель машины ('2000г', 'I-PRODUCT')
5. Определил марку машины как бренд ('2752', 'B-PRODUCT')
6. Определил глагол как марку машины ('Продам', 'B-PRODUCT')